Questo programma crea una lista di stocks che rispetteranno i seguenti criteri: un volume minimo, un prezzo minimo e determinate candele nell'ultima giornata. In primis, il volume minimo ci permette di filtrare le nostre azioni in base al grado di liquidità desiderato (un volume più alto ci permette di analizzare mercati più liquidi); il prezzo minimo, impostato a 60 come default, ci consentirà di trattare azioni con una percentuale più alta di attori istituzionali (infatti, le share vendute a prezzi molto bassi permettono minori barriere all'entrata per i trader retail). Infine, il terzo parametro più importante è quello che tratta l'ultima candela giornaliera, infatti, la lista finale conterrà solo le azioni che hanno avuto una candela giornaliera con una chiara direzione di mercato.

Bisogna operare con il seguente programma solo prima dell'apertura dei mercati statunitensi o in seguito alla chiusura. Ad esempio, se vogliamo analizzare le candele della giornata del 21 Maggio, allora dovremo attivare questo programma il 22 Maggio prima dell'aperture dei mercati (prima delle 15:30 italiane).

"""
DISCLAIMER

Il seguente codice è fornito unicamente a scopo educativo e di analisi. Non rappresenta in alcun modo un consiglio finanziario, una raccomandazione d'investimento, o un'offerta di acquisto o vendita di strumenti finanziari. L'utente è pienamente responsabile delle proprie decisioni d'investimento e delle eventuali perdite che possono derivare dall'uso di questo codice.

Si consiglia vivamente di consultare un consulente finanziario qualificato prima di intraprendere qualsiasi attività di trading o investimento. L'autore del codice declina ogni responsabilità per eventuali perdite o danni derivanti dall'uso di questo codice.

UTILIZZANDO QUESTO CODICE, L'UTENTE ACCETTA DI ESSERE IL SOLO RESPONSABILE PER QUALSIASI DECISIONE FINANZIARIA PRESA E RINUNCIA A QUALSIASI PRETESA LEGALE CONTRO L'AUTORE.

"""

di Vincenzo Sguera

In [ ]:
#In primis, dobbiamo importare i seguenti pacchetti python
#ATTENZIONE, prima di poterli utilizzare bisogna installarli
import pandas as pd
from tradingview_ta import TA_Handler, Interval, Exchange
import yfinance as yf
from datetime import datetime, timedelta


#questo è il link che contiene la documentazione del pacchetto tradingview_ta
'''https://tvdb.brianthe.dev/'''

#il seguente link è l'origine del file su cui costruiremo la nostra variabile df. Dopo aver aperto il link basta cliccare su "Dowload CVS"
'''https://www.nasdaq.com/market-activity/stocks/screener'''



#dobbiamo importare ed utilizzare il seguente file csv in modo da ottenere la lista di asset presenti sul NASDAQ
#tra le parentesi tonde bisogna inserire il path: dopo aver fatto il download del file CSV dal link prima esposto, basta cliccare col tasto destro sul file scaricato e poi premere "copia percorso".
#in seguito dovremo solo incollare il path tra le parentesi tonde
df = pd.read_csv("C:\\Users\\vinci\\Downloads\\nasdaq_screener_1716198987029.csv")



#dobbiamo impostare le due condizioni elencate all'inizio:
min_price = 60              #questa variabile rappresenta il prezzo minimo dell'asset
min_vol = 2000000           #questa variabile rappresenta il volume minimo dell'ultima giornata

#la seguente funzione ci aiuterà a pulire le colonne, in particolare ci permette di trasformare i dati di interesse in dati numerici
def clean_numeric_column(column):
    if df[column].dtype == 'object':
        df[column] = pd.to_numeric(df[column].str.replace('$', '').str.replace(',', ''), errors='coerce')
    return df[column]

#applichiamo la funzione alle colonne di interesse
df['Last Sale'] = clean_numeric_column('Last Sale')
df['Volume'] = clean_numeric_column('Volume')

#verifichiamo che tipologia di dati sono presenti nelle varie colonne, Last Sale e Volume dovrebbero essere rispettivamente float64 e int64
print(df.dtypes)

#creiamo la variabile filtered_df, impostando due condizioni: min_vol e min_price. Dunque, questa variabile creerà una "lista" di azioni che rispettino questi due criteri
filtered_df = df[(df['Last Sale'] > min_price) & (df['Volume'] > min_vol)]

#tickers è una variabile molto importante, racchiude i tickers che abbiamo filtrato
tickers = filtered_df['Symbol']

#possiamo visualizzare poi la lista 
print(tickers)



In [ ]:
#we set the interval we are interested in
#impostiamo due variabili importanti: start_date e end_date. Ci saranno utili in seguito quando dovremo importare da internet le informazioni sulla candela di interesse
start_date = datetime.now() - timedelta(days=4)
end_date = datetime.now()

#impostiamo in anticipo la nostra lista di interesse che conterrà tutti i ticker di interesse. La compileremo in seguito
list_stocks = []


#ora, per ogi ticker filtrato in precedenza, dobbiamo importare le informazioni sulla candela
for ticker in tickers:
#la variabile chiamata "data" è formata da 4 parametri importanti: ticker, data di inizio e data di fine, e l'intervallo di candele di interesse. 
#Dunque, data è la variabile che contiene le informazioni delle candele dell'intervallo compreso tra start_date e end_date dei ticker nella lista.
#E' importante soffermarsi sul parametro 'interval='1d'', infatti questo codice è impostato per filtrare le azioni in base all'ultima candela giornaliera, possiamo tranquillamente cambiare l'intervallo (ad esempio '15m)
    data = yf.download(ticker, start=start_date, end=end_date, interval='1d')
#se non riusciamo ad importare le informazioni adeguate da una determinata azione allora proseguiamo analizzando le altre
    if data.empty:
        print(f'There are no data for {ticker} in the interval designed')
        continue
        
#we create a new variable called yesterday, it will contains the data of the last candle (yesterday's candle): High, Low, Open, Close, Adj Close and Volume
#creiamo una variabile chiamata "yesterday", conterrò le informazioni dell'ultima candela giornaliera: High, Low, Open, Close, Adj Close e Volume
#inoltre possiamo cambiare l'indice presente in data.iloc[i], ad esempio se vogliamofiltrare le azinìoni in base alla candela giornaliera di due giorni fa basta inserire il 2 al posto del numero 1
    yesterday = data.iloc[-1]
    

#inoltre dobbiamo impostare le variabili principali utili per regolare i criteri che deve rispettare l'ultima candela
    aa = yesterday['Open']
    bb = yesterday['Close']
    cc = yesterday['High']
    dd = yesterday['Low']
    ee =yesterday['Volume']

#queste sono le operazioni principali che ci interessano
    a = cc==bb
    b = dd==aa
    c = cc==aa
    d = dd==bb
    e = aa-dd
    f = bb-aa
    g = aa-bb
    h = cc-aa
    i = bb-dd
    j = cc-bb


    

#infine, le linee di codice mostrate sotto rappresentano il compilatore della nostra lista. Attraverso le condizioni prestabilite, la nostra lista si aggiornerà aggiungendo le azioni filtrate anche con le candele di interesse
    if a or j <= f/2:
        if e >= 2*f:
            print(f'The stock which ticker is {ticker} satisfies the criteria')
            list_stocks.append(ticker)

    elif b or e <= f/2:
        if j >= 2*f:
            print(f'The stock which ticker is {ticker} satisfies the criteria')
            list_stocks.append(ticker)

    elif c or h <= g/2:
        if i >= 2*g:
            print(f'The stock which ticker is {ticker} satisfies the criteria')
            list_stocks.append(ticker)

    elif d or i <= g/2:
        if h >= 2*g:
            print(f'The stock which ticker is {ticker} satisfies the criteria')
            list_stocks.append(ticker)
    else:
        continue


La variabile yesterday è molto importante, nel programma di default: yesterday = data.iloc[-1]. Se inseriamo -1 tra le parentesi quadre vuol dire che il nostro programma valuterà l'ultima candela disponibile delle azioni analizzate. Se per esempio avessimo inserito -2 al posto del valore di defaut allora il programma avrebbe analizzato la penultima candela disponibile.

In [ ]:
#Ecco la lista delle azioni da considerare
print(list_stocks)

CONTATTI e SUPPORTO: 
vincenzosguera150@gmail.com